# 그림 생성


In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg=-0.4.3

# 훈련된 모델 가져오기

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

In [ ]:
%cd stylegan2-ada-pytorch/
!wget -nc https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
%cd ..

## 그림 생성, 저장

In [ ]:
import torch, pickle
import functools

from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from tqdm import tqdm

In [ ]:
from torchvision import models, transforms
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
import sys
import os
from os.path import isfile
import json
import argparse
from sklearn.svm import LinearSVC

In [ ]:
%cd stylegan2-ada-pytorch

In [ ]:
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)

with open('ffhq.pkl', 'rb') as f:
  G = pickle.load(f)['G_ema'].to(device)
if device_name == "cpu":
  G.synthesis.forward = functools.partial(G.synthesis.forward, force_fp32=True)

%cd ..

In [ ]:
!mkdir test_UnlabeledImages
!mkdir test_ImageLatentCodes

In [ ]:
N = 10

for k in tqdm(range(N)):
  with torch.inference_mode():
    z = torch.randn([1, G.z_dim]).to(device)
    torch.save(z, f'test_ImageLatentCodes/tensor{k}.pt')
    Image = (0.5*(G(z, None)[0]+1)).cpu().detach().numpy().clip(0,1)
    plt.imsave(f'test_UnlabeledImages/img{k}.jpg', image.transpose(1,2,0))

In [ ]:
image = image.transpose(1,2,0)

In [ ]:
plt.imshow(image)
plt.axis('off')

Discriminator 불러오기

In [ ]:
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
print(device)

In [ ]:
class EfficientNetB0(nn.Module):
  def __init__(self):
    super(EfficientNetB0, self).__init__()
    self.model = models.efficientnet_b0(pretrained=True)
    self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
    self.head = nn.Sequential(
        nn.Linear(1280, 1024),
        nn.BatchNorm1d(1024),
        nn.Dropout(0.2),
        nn.Linear(1024, 256),
        nn.BatchNorm1d(256),
        nn.Dropout(0.2),
        nn.Linear(256, 40)
    )

  def forward(self, x):
    x = self.model.features(x)

    x = self.avgpool(x)
    x = x.view(x.shape[0], -1)
    x = self.head(x)
    return x

In [ ]:
!wget -nc https://github.com/GurvanR/GANSpace-Reimplementation/raw/main/atclas2.pt

In [ ]:

net = EfficientNetB0()
net.load_state_dict(torch.load('atclas2.pt'))
net = net.to(device)
net.eval()

classification

In [ ]:
!gdown --id 1hY3zxdC9gf4-R4xfDONiJPpaOddq2yr

In [ ]:
df = pd.read_csv('list_attr_celeba.csv')

att_names = np.array(list(df.columns[1:]))

In [ ]:
batch = torch.from_numpy(image).float().cuda().unsqueeze(0).permute(0,3,1,2)
batch.shape

In [ ]:
out = net(batch).detach().cpu()


In [ ]:
torch.sigmoid(out.view(-1))>0.5

In [ ]:
for i,u in enumerate(torch.sigmoid(out.view(-1))>0.5):
  if u:
    print(att_names[i])

In [1]:
def run_classification(dataset_path, output_path, proportion, batch_size=10):
  if not os.path.exists(output_path):
    os.makedirs(output_path)
  if not os.path.isdir(output_path):
    raise ValueError("Output path muse be a directory, not overwriting existing file")

  filenames = [f for f in os.listdir(dataset_path) if isfile(os.path.join(dataset_path, f))]
  filenames_and_scores = [[] for _ in range(40)]
  totensor = transforms.ToTensor()

  for i in tqdm(range(0, len(filenames), batch_size)):
    batch = torch.zeros((batch_size, 3, 1024, 1024))
    batch_filenames = filenames[i:min(i+batch_size, len(filenames))]
    for j, f in enumearte(batch_filenames):
      path = os.path.join(dataset_path, f)
      if not isfile(path):
        continue
      img = Image.open(path)
      batch[j] = 255+totensor(img)
      img.close()
    batch = batch.to(device)

    # batch 내용을 network에 저장하기
    with torch.inference_mode():
      out = net(batch).cpu()
    for j, f in enumerate(batch_filenames):
      for att in range(40):
        filenames_and_scores[att].append((f, out[j, att].item()))
  # 결과를 json 파일에 저장하기
  for att in range(40):
    with open(os.path.join(output_path, f"att{att}_scores.json"), "w") as outfile:
      json.dump(dict(filenames_and_scores[att]), outfile)

    # classes를 생성하고 연관성 있는 특성들로 구분하기!
    filenames_and_scores[att].sort(key=lambda p: p[1])
    num_top = int(proportion*len(filenames)/100)


In [ ]:
proportion = 2

dataset_path = 'test_UnlabeledImages'
output_path = 'Labels'

run_classification(dataset_path, output_path, proportion)

In [ ]:
run_classification(dataset_path, output_path, proportion)

## Retrieving labeled latent vectors

Double-cliquez

In [ ]:
attribute_numbers=[np.where(att_names==u)[0][0] for u in ['Smiling', 'Young', 'Eyeglasses', 'Male', 'Mustache']]

In [ ]:
attribute_numbers

In [ ]:
latent_vectors_path = 'test_ImageLatentCodes'

In [ ]:
for attribute_number in attribute_numbers:
  labels_path = f'Labels/att{str(attribute_number)}_labels.json'
  with open(labels_path, 'r') as f:
    labels_dict = json.load(f)

  num_samples = len(labels_dict)
  dim = 512
  Z = torch.zeros((num_samples, dim)).to(device)
  y = np.zeros(num_samples, dtype=int)

  for i, (filename, label) in tqdm(emumerate(labels_dict.items())):
    sample_number = int(filename[3:-4])
    tensor_path = f'test_ImageLatentCodes/tensor{sample_number}.pt'
    Z[i] = torch.load(tensor_path, map_location=device)
    y[i] = label

  # SVM

  Z_forSVM = z.cpu().numpy()

  linear_svm_Z = LinearSVC()
  linear_svm_z.fit(Z_forSVM, y)
  # orthogonal vector를 얻기 - svm을 통해 만들어진 하이퍼 평면의 벡터
  class_direction_Z = torch.Tensor(linear_svm_Z.coef_[0])

  Z_intercept = torch.Tensor(linear_svm_Z.intercept_)

  W = G.mapping(Z, None)

  W_for_SVM = W[:,0,:].cpu().numpy()

  linear_svm_W = LinearSVC()
  linear_svm_W.fit(W_for_SVM, y)

  class_direction_W = torch.Tensor(linear_svm_W.coef_[0])

  W_intercept = torch.Tensor(linear_svm_W.intercept_)

  if not os.path.exists('SVM_vectors'):
    os.makedirs('SVM_vectors')

  torch.save(class_direction_Z, f'SVM_vectors/class_direction_Z_on_3k_att{attribute_number}.pt')
  torch.save(class_direction_W, f'SVM_vectors/class_direction_W_on_3k_att{attribute_number}.pt')

  torch.save(Z_intercept, f'SVM_vectors/Z_intercept_on_3k_att{attribute_number}.pt')
  torch.save(W_intercept, f'SVM_vectors/W_intercept_on_3k_att{attribute_number}.pt')

